In [41]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [42]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [43]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(50000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [44]:
df

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard
review_id,,,,,,,,,,,,
R2V46CPENK0FRB,B009GDBNF8,Starbucks Breakfast Blend K-Cup for Keurig Bre...,5,0,0,N,Y,great,great. speedy delivery. great coffee,0.400000,0.375000,NaN
R1LWTUWRMSVT77,B002C37TM0,"Prince of Peace Tea, Oolong, 100-Count",4,0,0,N,Y,Four Stars,Four Stars. Great but very small box for the p...,0.237500,0.635000,NaN
R2Y23W89VV4F5N,B002EDDA2Q,Reese's Dark Chocolate Peanut Butter Cup Minia...,5,0,0,N,Y,I like them too,I like them too. My husband loves these- he co...,-0.002083,0.116667,NaN
R1W4T4ONMCCDKW,B003D4GVM6,"Stash Tea Decaf Chai Spice Black Tea, 100 Coun...",5,0,0,N,Y,Love this tea.,Love this tea.. My wife and I love this tea. ...,0.583333,0.750000,NaN
R3B0RFSZUPZJJG,B0017K5HH8,Big Train Chai Tea Chocolate 3.5 lb bulk,5,0,0,N,Y,Five Stars,Five Stars. my daughter's favorite,0.500000,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
R2ANDIEFJP1QTT,B003GS053U,Beveri Organic Golden Flaxseed Whole 16 oz,4,0,0,N,Y,Great product,Great product. These flaxseeds are wonderful o...,0.821429,0.883929,NaN
R675KX9RKDV2L,B00CHTSWB6,Siggis Skyr Orange and Ginger Icelandic Style ...,5,3,3,N,N,A delicious high protein ginger-y treat!,A delicious high protein ginger-y treat!. The ...,0.127340,0.508036,NaN
R3EJ1M8UML6V9Z,B004L045D8,"Jelly Belly Beans, Fruit Bowl, 1 Pound",4,0,0,N,Y,But after realizing that it has all my favorit...,But after realizing that it has all my favorit...,0.091898,0.317593,NaN


In [52]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [53]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

neg_reviews["tokened_review"] = np.nan
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    neg_reviews.loc[index, "tokened_review"] = ' '.join(new_text)
neg_reviews["tokened_review"][0]

<ipython-input-53-5557a774945a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = np.nan
C:\Users\MattW\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


'yuck thought purchas christma time costco purchas valentin day aw tast like cocoa cover lard dont care price id rather delici tast like one costco dont anymor dont know anybodi say tast delici tast'

In [54]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale"]
for word in hazardous_words:
    neg_reviews.loc[[word in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

In [55]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

432

In [56]:
non_hazardous = neg_reviews.query("health_hazard != 1 and polarity < 0", engine="python").sample(num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\Users\MattW\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [57]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=200)
X = matrix.fit_transform(reviews).toarray()
X

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.4029168,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [59]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [60]:
#from sklearn.naive_bayes import GaussianNB
#classifier = GaussianNB()

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8796296296296297


In [61]:
trial = neg_reviews.sample(100)

In [62]:
matrix.get_feature_names()

['actual',
 'almost',
 'also',
 'amazon',
 'ani',
 'anoth',
 'anyth',
 'around',
 'arriv',
 'aw',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'batch',
 'bean',
 'becaus',
 'befor',
 'best',
 'better',
 'big',
 'bit',
 'bottl',
 'bought',
 'box',
 'brand',
 'broken',
 'buy',
 'came',
 'candi',
 'cant',
 'chocol',
 'coffe',
 'come',
 'compani',
 'contain',
 'cook',
 'cooki',
 'corn',
 'could',
 'cup',
 'date',
 'day',
 'didnt',
 'differ',
 'disappoint',
 'disgust',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'end',
 'enough',
 'even',
 'ever',
 'expect',
 'expens',
 'expir',
 'find',
 'first',
 'flavor',
 'food',
 'found',
 'free',
 'fresh',
 'get',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'ha',
 'half',
 'hard',
 'high',
 'horribl',
 'howev',
 'id',
 'im',
 'ingredi',
 'item',
 'ive',
 'know',
 'label',
 'last',
 'leav',
 'less',
 'like',
 'littl',
 'local',
 'long',
 'look',
 'lot',
 'love',
 'made',
 'make',
 'mani',
 'may',
 'mayb',
 'might',
 'milk',
 'mix',
 

In [63]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier.predict(trial_X)
trial_y

array([0., 0., 1., ..., 0., 1., 0.])

In [64]:
neg_reviews["health_hazard"] = trial_y

<ipython-input-64-56d181f868ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["health_hazard"] = trial_y


In [65]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R2XW5VDHMEQ6J2,B0008IT4OM,Chocmod Truffettes de France Natural Truffles,1,14,14,N,Y,YUCK,YUCK. I thought these were the same as I purch...,0.200000,0.625000,0.0,yuck thought purchas christma time costco purc...
R3DB756YWCMDQ8,B006J8M85M,Dr. Fuhrman's Mediterranean Stone Pine Nuts,3,1,2,N,Y,ok product,ok product. the freshness and flavor is good. ...,0.025000,0.325000,0.0,ok product fresh flavor good quaniti price bit...
R1RDBG2GUOQFUO,B00F4WIQ96,Wine Country Gift Baskets Sympathy Basket,1,11,11,N,Y,Food was Stale!,Food was Stale!. I sent this basket to my fami...,-0.032576,0.407407,1.0,food wa stale sent thi basket famili grandfath...
R1VZ5J3UQ3MBJI,B0001N7RLQ,E-Fa Brand Bubble Tea Black Tapioca Pearl 2.2LB,1,7,8,N,Y,Product advertised is NOT what they shipped. T...,Product advertised is NOT what they shipped. T...,0.002324,0.323076,0.0,product advertis ship suck updat 9313 cook bol...
R34ICMO0UAZFSG,B000H27PB8,Ritter Sport,3,2,3,N,Y,What happened to the price?,What happened to the price?. I love this choco...,0.028125,0.506719,0.0,happen price love thi chocol bar happen price ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R3TVRXQSUTL7TC,B004G7VL88,12 Pack 2 Oz Bottles Quick Beverages Sleep,1,1,1,N,Y,Liquid Sleep,"Liquid Sleep. This did not work for me, but it...",-0.125000,0.375000,0.0,liquid sleep thi work may peoplebr im one sleep
RJXNE68NQR369,B003AIF4Q4,High5 Zero-Berry,2,1,1,N,N,Not for me,Not for me. For now 2 stars.<br /><br />I am a...,0.045833,0.444097,0.0,2 starsbr avid road cyclist also type 2 diabet...
R3MG7X8DSLOX2W,B000EDK814,"Raw Revolution Organic Live Food Bars, 1.8-Oun...",1,6,11,N,Y,Raw food,Raw food. Not to my liking. I usually like hea...,-0.109375,0.187500,0.0,raw food like usual like health food thi wa li...


In [66]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R1RDBG2GUOQFUO,B00F4WIQ96,Wine Country Gift Baskets Sympathy Basket,1,11,11,N,Y,Food was Stale!,Food was Stale!. I sent this basket to my fami...,-0.032576,0.407407,1.0,food wa stale sent thi basket famili grandfath...
R3T5EB07Z681A2,B005MSH1NY,SodaStream Natural Syrup,1,2,2,N,Y,disgusting.,"disgusting.. Think listerine, mixed with furni...",0.046296,0.223718,1.0,disgust think listerin mix furnitur polish str...
R18AXSH45A6E77,B000GB0TMW,Matcha Green Tea Powder 1 oz loose tea sample,1,3,3,N,N,This matcha tastes not quite like matcha. It l...,This matcha tastes not quite like matcha. It l...,0.000000,0.312500,1.0,thi matcha tast quit like matcha leav powderi ...
RCEXTIDQ0IL7E,B002GWMEO0,Green Unroasted Whole Bean Coffee,3,0,0,N,Y,Just Okay,Just Okay. These beans were just okay. Nothing...,0.037202,0.532440,1.0,okay bean okay noth special mayb fresh purveyo...
R2QPQ5DM7V06H8,B005ZGJLP4,Black Jelly Beans - Licorice Flavor - 2 Lbs.,2,0,0,N,N,weak flavor/arrived crushed,weak flavor/arrived crushed. These jelly beans...,0.126111,0.496111,1.0,weak flavorarriv crush jelli bean arriv pad en...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R1X3YSMZJ6NAQX,B00AYQD3L8,SUNKIST Fruit Gems Wrapped (5 pound bag),1,3,4,N,Y,All sugar no fruit flavor.,All sugar no fruit flavor.. I have bought frui...,0.259375,0.450000,1.0,sugar fruit flavor bought fruit gem mani time ...
RQLGUKWE5S0YQ,B003WFJW40,Nescafe Taster Choice Hazelnut Coffee 20 Singl...,1,3,4,N,Y,Not fresh,Not fresh. I purchased this product at the end...,0.070370,0.324691,1.0,fresh purchas thi product end novemb veri blan...
R1UXCL9XGV7XAR,B00250UF6C,"WillPowder Coconut Milk Powder, 16-Ounce Jar",3,18,24,N,N,Ingredients and nutritional values would be he...,Ingredients and nutritional values would be he...,0.119464,0.637083,1.0,ingredi nutrit valu would help willpowd websit...


In [83]:
#LDA
data_filtered = dangerous['tokened_review'].values
data_filtered

C:\Users\MattW\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['food wa stale sent thi basket famili grandfath die live 3000 mile away want send someth ive order item wine countri gift basket mani times…but rel told know food wa stale ined im sure happen it…but thi veri upset disappoint',
       'disgust think listerin mix furnitur polish strain thru loaf breadbr purchas thi wife like regular flavor ss ginger ale albeit doubl strength fan canada dri fact canadian relev relay reaction productbr tri differ strength mix start recommend dose onli got offensivebr option zero star explor option returnrefund',
       'thi matcha tast quit like matcha leav powderi film thi matcha tast quit like matcha leav powderi film mouth think tast someth along line mochiko flour mix matcha mayb make weigh 1lb doe tast like matcha wa onli thing packag',
       ...,
       'ingredi nutrit valu would help willpowd websitebr coconut milk powder uniqu product spraydri coconut milk shelf stabl easi use thi product come onli pure freshli press coconut milk combin nat

In [84]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(str(sentence)))

data_words = list(sent_to_words(data_filtered))

C:\Users\MattW\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [85]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
pprint.pprint(corpus[:5])

[[(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 2),
  (26, 1),
  (27, 2),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 2),
  (33, 1),
  (34, 1)],
 [(27, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 2),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1)],
 [(24, 1),
  (27, 2),
  (32, 1),
  (52, 3),
  (55, 1),
  (57, 1),
  (73, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1),
  (83, 

C:\Users\MattW\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [86]:
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       minimum_probability=0.1,
                                       workers=3,
                                       passes=2)
# Print the Keyword in the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

C:\Users\MattW\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.027*"wa" + 0.020*"thi" + 0.011*"tast" + 0.011*"good" + 0.010*"like" + '
  '0.009*"would" + 0.009*"flavor" + 0.009*"tri" + 0.008*"price" + 0.007*"one"'),
 (1,
  '0.014*"thi" + 0.014*"like" + 0.009*"flavor" + 0.008*"wa" + 0.008*"much" + '
  '0.007*"veri" + 0.007*"ha" + 0.007*"good" + 0.006*"one" + 0.006*"’"'),
 (2,
  '0.025*"wa" + 0.020*"thi" + 0.019*"like" + 0.012*"tast" + 0.007*"product" + '
  '0.007*"would" + 0.007*"tri" + 0.006*"mix" + 0.006*"order" + 0.006*"packag"'),
 (3,
  '0.028*"thi" + 0.018*"tast" + 0.014*"like" + 0.014*"wa" + 0.008*"good" + '
  '0.008*"someth" + 0.008*"product" + 0.008*"chocol" + 0.007*"flavor" + '
  '0.006*"tri"'),
 (4,
  '0.019*"thi" + 0.010*"wa" + 0.009*"tast" + 0.009*"like" + 0.007*"one" + '
  '0.007*"use" + 0.005*"chocol" + 0.005*"tri" + 0.005*"make" + '
  '0.005*"product"'),
 (5,
  '0.025*"wa" + 0.025*"thi" + 0.015*"tast" + 0.011*"tea" + 0.009*"like" + '
  '0.009*"good" + 0.008*"box" + 0.008*"would" + 0.007*"one" + 0.006*"someth"'),
 (6,
  '0.0

In [87]:
from itertools import chain
lda_corpus = lda_model[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
scores = list(chain(*[[score for topic_id,score in topic] for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)


C:\Users\MattW\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [88]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
data_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(data_vis)

C:\Users\MattW\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
